In [85]:
import requests
import pandas as pd
from pprint import pprint
import sqlite3
from html import unescape
from bs4 import BeautifulSoup
import re
session = requests.session()
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from collections import Counter
from sklearn.metrics import accuracy_score

*Ищем положительные отзывы и очищаем их от тегов*

In [33]:
url_good = f'https://www.kinopoisk.ru/film/688/reviews/?status=good&ord=date&rnd=1601992343&perpage=50'
req = session.get(url_good, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
good = soup.find_all('span', {'itemprop': 'reviewBody'})

[<span class="_reachbanner_" itemprop="reviewBody">Второй фильм о Гарри Поттере, поставленный по книге Дж. Роулинг, 'Гарри Поттер и тайная комната' снятый режиссером Крисом Коламбусом сумел удержать тот высокий уровень, на котором оказался первый фильм 'Гарри Поттер и философский камень'. А удержать уровень чаще сложнее, чем его достигнуть.<br/>
<br/>
 В фильме действие разворачивается, как в романе, что Гарри живет у семьи Дурслей, которые по прежнему не дают ему родительской любви, добра, хорошего воспитания. Гарри желает вернуться в  школу Хогвардс, продолжить учебу, там его друзья, там он оценен по призванию, может развиваться дальше,  а не загнан в комнату, где один.  <br/>
<br/>
 Несмотря на предостережение Добби не ехать, Гарри вместе с друзьями отправляется в путь. В волшебную школу и дорога, полная волшебства, приключений, что уже затягивает юного зрителя. <br/>
<br/>
 Учеба началась. Среди преподавателей по-прежнему добрый и хороший Дамблдор, холодный, жесткий, но при этом об

In [134]:
cleanr = re.compile('<.*?>')
cleangood = re.sub(cleanr, '', str(good))

<class 'str'>


*Ищем отрицательные отзывы и очищаем их от тегов*

In [46]:
url_bad = f'https://www.kinopoisk.ru/film/407636/reviews/?status=bad&ord=date&rnd=1601997467&perpage=50&page=2'
req = session.get(url_bad, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
bad = soup.find_all('span', {'itemprop': 'reviewBody'})
print(bad)

[<span class="_reachbanner_" itemprop="reviewBody">Первая мысль после просмотра 'Ну вот и закончилось'. И я думал вовсе не о культе Гарри Поттера. Сказка про волшебника будет сводить с ума школьников начальных классов ещё очень долго. Почему именно их? Влюбиться в этот мир в более позднем возрасте не возможно. Так что если вы в своё время пропустили книги/фильмы о «мальчике, который выжил», то сейчас, когда вы выросли (при условии что вы выросли) вы ничего не поймёте.<br/>
<br/>
 Так что же закончилось? Закончился фильм. Просто очередной фильм про Гарри Поттера. Ожидалось, что фильм поставит точку. Что будет грандиозная концовка, какая должны быть у серии фильмов, которые выходят с 2001 года. А концовка получилась такая, словно в следующем году у фильма выйдет очередное продолжение. <br/>
<br/>
 Первым делом хочется отметить ужасное 3D. Что оно есть, что его нет - разница только в трёхмерных субтитрах и неудобных очках. Идти на фильм в 3D совершенно не имеет смысла.<br/>
<br/>
 Из актё

In [136]:
cleanr = re.compile('<.*?>')
cleanbad = re.sub(cleanr, '', str(bad))

<class 'str'>


*Ищем тестовые отзывы, очищаем их от тегов, кладем в список*

In [48]:
url_test = f'https://www.kinopoisk.ru/film/1143242/reviews/'
req = session.get(url_test, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
test = soup.find_all('span', {'itemprop': 'reviewBody'})

[<span class="_reachbanner_" itemprop="reviewBody">'Умение соответствовать необходимо в каждом аспекте жизни, не говоря уже о гардеробе. Для каждого наряда есть сезон, а для сезона стратегия.'<br/>
<br/>
Очаровательные актёры и живой мир преступной Англии. Этот фильм нужно прочувствовать, как английский чай, иначе есть риск упустить вагон деталей. С чувством юмора у фильма всё в порядке, в меру остроумно и смешно. Ведь джентльмен должен знать меру. Условно фильм можно поделить на две части - вводную и динамичную. Первая 'часть' фильма состоит буквально из пересказа сценария к фильму, кто-нибудь заметил, насколько грамотно Гай Ричи только что сломал четвёртую стену? Она идёт медленно и размерено, будто сам режиссёр пытается с нами поговорить. если внимательно вслушиваться в диалоги между Рэем и Флетчером можно многое узнать непосредственно до того, как режиссёр прямым текстом объявит нам это. Во второй 'части' события начинают развиваться динамично, диалоги становятся более напряжёнными

In [150]:
test_sp = str(test).split('<span class="_reachbanner_" itemprop="reviewBody">')
for i in test_sp:
    cleant = re.compile('<.*?>')
    cleantest = re.sub(cleant, '', str(i))
    cleanre = re.compile('[^а-яёА-ЯЁ ]')
    cleantest = re.sub(cleanre, '', str(i))
del test_sp[0]

'Умение соответствовать необходимо в каждом аспекте жизни, не говоря уже о гардеробе. Для каждого наряда есть сезон, а для сезона стратегия.'<br/>
<br/>
Очаровательные актёры и живой мир преступной Англии. Этот фильм нужно прочувствовать, как английский чай, иначе есть риск упустить вагон деталей. С чувством юмора у фильма всё в порядке, в меру остроумно и смешно. Ведь джентльмен должен знать меру. Условно фильм можно поделить на две части - вводную и динамичную. Первая 'часть' фильма состоит буквально из пересказа сценария к фильму, кто-нибудь заметил, насколько грамотно Гай Ричи только что сломал четвёртую стену? Она идёт медленно и размерено, будто сам режиссёр пытается с нами поговорить. если внимательно вслушиваться в диалоги между Рэем и Флетчером можно многое узнать непосредственно до того, как режиссёр прямым текстом объявит нам это. Во второй 'части' события начинают развиваться динамично, диалоги становятся более напряжёнными, происходит много экшена. Те, кто уснул во время д

*Приводим текст к нижнему регистру и токенизируем в функции, а затем применяем ее для положительных и отрицательных комментариев* 

In [127]:
def tokenizing(text):
    text = text.lower()
    lemmas = []
    token = word_tokenize(text)
    for word in token:
        an = morph.parse(word)
        first = an[0]
        lemma = first.normal_form
        lemmas.append(lemma)
    return(lemmas)

In [194]:
def low_freq(text):
    freqlist = Counter()
    for word in text:
        if word.isalpha():
            freqlist[word] += 1
    d = dict(freqlist.most_common())
    for w in d:
        if d.get(w)<=2:
            text.remove(w)
    return (text)

In [141]:
tokbad = tokenizing(cleanbad)
tokgood = tokenizing(cleangood)

In [195]:
low_freq(tokbad)
low_freq(tokgood)

['[',
 'два',
 'фильм',
 'о',
 'гарри',
 'поттер',
 ',',
 'поставить',
 'по',
 'книга',
 'дж',
 '.',
 'роулинга',
 ',',
 "'гарри",
 'поттер',
 'и',
 'тайный',
 'комната',
 "'",
 'снять',
 'режиссёр',
 'крис',
 'коламбус',
 'суметь',
 'удержать',
 'тот',
 'высокий',
 'уровень',
 ',',
 'на',
 'который',
 'оказаться',
 'один',
 'фильм',
 "'гарри",
 'поттер',
 'и',
 'философский',
 'камень',
 "'",
 '.',
 'а',
 'удержать',
 'уровень',
 'сложный',
 ',',
 'чем',
 'он',
 'достигнуть',
 '.',
 'в',
 'фильм',
 'действие',
 ',',
 'как',
 'в',
 'роман',
 ',',
 'что',
 'гарри',
 'жить',
 'у',
 'семья',
 'дурслить',
 ',',
 'который',
 'по',
 'прежний',
 'не',
 'давать',
 'он',
 'любовь',
 ',',
 ',',
 'хороший',
 '.',
 'гарри',
 'желать',
 'вернуться',
 'в',
 'школа',
 'хогвардс',
 ',',
 'продолжить',
 'учёба',
 ',',
 'там',
 'он',
 'друг',
 ',',
 'там',
 'он',
 'по',
 ',',
 'мочь',
 'далёкий',
 ',',
 'а',
 'не',
 'в',
 'комната',
 ',',
 'где',
 'один',
 '.',
 'несмотря',
 'на',
 'добби',
 'не',
 'еха

*Делим комментарии на 2 множества*

In [196]:
good_comments = set(tokgood)
bad_comments = set(tokbad)
only_bad = bad_comments.difference(good_comments)
only_good = good_comments.difference(bad_comments)

*Создаем список, в который добавляем информацию о тестовых комментариях: 1 - положительный, 0 - отрицательный*

In [197]:
lis = []

In [198]:
def good_or_bad(comment):
    g = 0
    b = 0 
    x = 0
    for i in comment:
        if i in only_good:
            g+=1
        elif i in only_bad:
            b+=1
    if g>=b:
        x = 1
    else:
        x = 0
    lis.append(x)
    return lis

In [199]:
for i in test_sp:
    good_or_bad(i)

*Сравниваем получившийся список с правильными ответом при помощи accuracy*

In [200]:
correct = [1, 1, 1, 1, 0, 0, 1, 1, 0, 1]
accuracy_score(lis, correct)

0.7

**Как можно улучшить программу:**
*1)Добавить визуализацию (облако) для слов, встречающихся только в положительных или отрицательных комментариях;*
*2)Избавиться от служебных слов*